### PyPI `openplugincore`
This is the meat of OpenPlugin, it contains all tools you need to interface with ChatGPT plugins as you do on ChatGPT Pro itself.
#### Watch out
Issues and concerns to look out for
- python version : `openplugincore` requires python version >= `3.10`
- `You exceeded your current quota` : If your run into an error suggesting `You exceeded your current quota` that could be for several reasons, refer to this [StackOverflow answer](https://stackoverflow.com/a/75898717/9622142) on how to resolve it.

## Quickstart
1. pip install requirements: Versioning is super super important

In [1]:
# imports
from openplugincore import openplugin_completion
import json
import openai

In [2]:
# Load the OpenAI key from a file
with open('../secrets/openAI_key', 'r') as file:
    openAI_key = file.read().replace('\n', '')

simplest way to use `openplugincore`

In [3]:
from openplugincore import openplugin_completion

messages = [
    {
        "role": "user",
        "content": "Review the following repository: https://github.com/recommenders-team/recommenders. "
    }
]


openplugin_completion_generation = openplugin_completion(
    openai_api_key = openAI_key,
    plugin_name="repo_radar",
    truncate = True, # Defaults to True. Truncates the plugin API response to ensure the LLM's token limit is not exceeded
    messages = messages,
    model = "gpt-3.5-turbo-0613",
    temperature = 0,
)

print(json.dumps(openplugin_completion_generation, indent=2))

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


generation:  text='' generation_info={'finish_reason': 'function_call'} message=AIMessage(content='', additional_kwargs={'function_call': {'name': 'getRepoReadmeByOwnerAndRepo', 'arguments': '{\n  "params": {\n    "owner": "recommenders-team",\n    "repo": "recommenders"\n  }\n}'}}, example=False)
request_chain name: getRepoReadmeByOwnerAndRepo, arguments: {"params": {"owner": "recommenders-team", "repo": "recommenders"}}
{
  "id": "chatcmpl-94x1beQCZGkripbFhzLrFIKZOCrRl",
  "object": "chat.completion",
  "created": 1710967231,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The repository \"recommenders\" is a project under the Linux Foundation of AI and Data. It aims to assist researchers, developers, and enthusiasts in prototyping, experimenting with, and bringing to production a range of classic and state-of-the-art recommendation systems.\n\nThe repository contains examples and best practic

For more nuanced use

In [4]:
from openplugincore import OpenPlugin

plugin = OpenPlugin("GifApi", openAI_key)

messages = [
    {
        "role": "user",
        "content": "show me a gif of a gangster cat"
    }
]

function_response = plugin.fetch_plugin(
    model="gpt-3.5-turbo-0613",
    messages = messages,
    truncate = True, # Truncates the plugin API response to ensure the LLM's token limit is not exceeded
    temperature=0,
)
print("function_response", function_response)

OpenPlugin_generation = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages= messages + [function_response],
    temperature = 0
)

print(json.dumps(OpenPlugin_generation, indent=2))

Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


generation:  text='' generation_info={'finish_reason': 'function_call'} message=AIMessage(content='', additional_kwargs={'function_call': {'name': 'getSearchResults', 'arguments': '{\n  "json": {\n    "query": "gangster cat"\n  }\n}'}}, example=False)
request_chain name: getSearchResults, arguments: {"json": {"query": "gangster cat"}}
function_response {'role': 'function', 'name': 'getSearchResults', 'content': '{"data": [{"url": "http://gph.is/1Y74zp8", "title": "Cat Spin GIF by Keanu Movie", "source": "", "username": "keanumovie", "alt_text": "", "embed_url": "https://media3.giphy.com/media/l2JJvDhbnoSU9e2hW/giphy.gif"}, {"url": "http://gph.is/1QhK6dO", "title": "cat kitten GIF by Romy", "source": "http://www.youtube.com/watch?v=KjEusWO6VPg", "username": "romy", "alt_text": "", "embed_url": "https://media1.giphy.com/media/CUFd8c0eovNio/giphy.gif"}, {"url": "http://gph.is/1oMzDy3", "title": "confused cat GIF by Keanu Movie", "source": "", "username": "keanumovie", "alt_text": "", "emb

and to be respectful to plugin APIs you can use `OpenPluginMemo`

In [5]:
from openplugincore import OpenPluginMemo

open_plugin_memo = OpenPluginMemo() # Stores the plugin's config in memory so that it does not need to call the API to fetch the config every time the plugin is initialize.
open_plugin_memo.init()

plugin = open_plugin_memo.get_plugin('repo_radar') # returns the plugin if it is already initialized, otherwise returns None
if plugin is None: # in this demo it returns None
    plugin = open_plugin_memo.init_plugin('repo_radar') # initializes the plugin

# the rest is the same as using the OpenPlugin class
messages = [
    {
        "role": "user",
        "content": "Review the following repository: https://github.com/recommenders-team/recommenders. What is some of the code in the examples directory?"
    }
]

function_response = plugin.fetch_plugin(
    model="gpt-3.5-turbo-0613",
    messages = messages,
    truncate = True, # Truncates the plugin API response to ensure the LLM's token limit is not exceeded
    temperature=0,
)

OpenPlugin_generation = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages= messages + [function_response],
    temperature = 0
)

print(json.dumps(OpenPlugin_generation, indent=2))

MEMO READY


Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


generation:  text='' generation_info={'finish_reason': 'function_call'} message=AIMessage(content='', additional_kwargs={'function_call': {'name': 'getRepoReadmeByOwnerAndRepo', 'arguments': '{\n  "params": {\n    "owner": "recommenders-team",\n    "repo": "recommenders"\n  }\n}'}}, example=False)
request_chain name: getRepoReadmeByOwnerAndRepo, arguments: {"params": {"owner": "recommenders-team", "repo": "recommenders"}}
{
  "id": "chatcmpl-94x2BxVZK9dUrS8GYzRkpp6Akl2MG",
  "object": "chat.completion",
  "created": 1710967267,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The examples directory in the repository contains Jupyter notebooks that demonstrate how to build recommendation systems using various algorithms. Some of the code examples in the examples directory include:\n\n1. [Prepare Data](examples/01_prepare_data): This directory contains notebooks that demonstrate how to prepare and 